In [ ]:
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git
pykeen.env()

In [4]:
import os
print(os.getcwd())
base_dir = f"{os.getcwd()}"

/Users/ferran/PhD/yaleCollab


# Explore PyKEEN and default data

In [2]:
import pandas as pd

from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.datasets.nations import NATIONS_TRAIN_PATH

tf = TriplesFactory.from_path(NATIONS_TRAIN_PATH)

print(NATIONS_TRAIN_PATH)
print(tf)

# training, testing = tf.split()

# result = pipeline(

#     training=training,

#     testing=testing,

#     model='TransE',

#     epochs=5,  # short epochs for testing - you should go higher

# )

# result.save_to_directory('doctests/test_unstratified_transe')

/usr/local/lib/python3.9/site-packages/pykeen/datasets/nations/train.txt
TriplesFactory(num_entities=14, num_relations=55, create_inverse_triples=False, num_triples=1592, path="/usr/local/lib/python3.9/site-packages/pykeen/datasets/nations/train.txt")


In [3]:
pd.read_csv('/usr/local/lib/python3.9/site-packages/pykeen/datasets/nations/train.txt', sep='\t')

,netherlands,militaryalliance,uk
0,egypt,intergovorgs3,usa
1,jordan,relbooktranslations,usa
2,poland,timesincewar,ussr
3,uk,negativebehavior,ussr
4,poland,relintergovorgs,uk
...,...,...,...
1586,cuba,ngoorgs3,usa
1587,usa,timesincewar,china
1588,china,treaties,burma
1589,netherlands,eemigrants,usa


Format data for PYKEEN is in the form of a TSV file with the (h,r,t) format.
THis means that the edges are indeed directed.

WEe can now take either one of two approaches:

- Use STRING database ass a proof of concept. All of these interactions will be considered as @signsallnig interactions. KG will end up having 2 types of edges: those from strign and those from cellchat
- Proper approach with the Functional INteraction network from reactome. Here interaction edges can be of different types, and string comprehension will have to be built to decode the direction column into individual triplets with the (h,r,t) format

## RPY2 cheatsheet

Using R from Python

In [15]:
import rpy2
import rpy2.situation
for row in rpy2.situation.iter_info(): # Print Rpy2 info
    print(row)

from rpy2.robjects.packages import importr
base = importr("base")
here = importr("here")

if here.here()[0] != base_dir:
    raise ValueError

rpy2 version:
3.4.5
Python version:
3.9.13 (main, Aug  7 2022, 01:33:23) 
[Clang 13.1.6 (clang-1316.0.21.2.5)]
Looking for R's HOME:
    Environment variable R_HOME: /usr/local/Cellar/r/4.2.1_2/lib/R
    Calling `R RHOME`: /usr/local/Cellar/r/4.2.1_2/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/local/lib/R/library/stats/libs/
R version:
    In the PATH: R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/usr/local/Cellar/r/4.2.1_2/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'icucore', 'dl', 'm', 'iconv']
  library_dirs:
  ['/usr/local/opt/gettext/lib', '/usr/local/opt/readline/lib', '/usr/local/opt/xz/lib', '/usr/local/lib', '/usr/local/opt/gettext/lib', '/usr/local/opt/readline/lib', '/usr/local/opt/xz/lib', '/usr/local/lib', '/usr/local/Cellar/r/4.2.1_2/lib/R/lib', '/usr/local/Cellar/pcre2/

In [ ]:
from rpy2.robjects.packages import importr
base = importr("base")
here = importr("here")

INTepi21 = base.readRDS(here.here("CRCTME_story/Integration_CRCTME-Aug21/RData",
"INTepi21_filt_2022.rds"))
INTepi21

Load R chunks

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
R.version.string

In [ ]:
%%R -i INTepi21
library(base)
library(tidyverse)
library(Seurat)
library(here)

INTepi21

# STRING + CellchatDB KGE

In [20]:
import rpy2
import rpy2.situation
for row in rpy2.situation.iter_info(): # Print Rpy2 info
    print(row)

from rpy2.robjects.packages import importr
base = importr("base")
here = importr("here")

if here.here()[0] != base_dir:
    raise ValueError

from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

rpy2 version:
3.4.5
Python version:
3.9.13 (main, Aug  7 2022, 01:33:23) 
[Clang 13.1.6 (clang-1316.0.21.2.5)]
Looking for R's HOME:
    Environment variable R_HOME: /usr/local/Cellar/r/4.2.1_2/lib/R
    Calling `R RHOME`: /usr/local/Cellar/r/4.2.1_2/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/local/lib/R/library/stats/libs/
R version:
    In the PATH: R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/usr/local/Cellar/r/4.2.1_2/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'icucore', 'dl', 'm', 'iconv']
  library_dirs:
  ['/usr/local/opt/gettext/lib', '/usr/local/opt/readline/lib', '/usr/local/opt/xz/lib', '/usr/local/lib', '/usr/local/opt/gettext/lib', '/usr/local/opt/readline/lib', '/usr/local/opt/xz/lib', '/usr/local/lib', '/usr/local/Cellar/r/4.2.1_2/lib/R/lib', '/usr/local/Cellar/pcre2/

Download cellchat DB with the following:

`wget https://github.com/sqjin/CellChat/raw/master/data/CellChatDB.mouse.rda`

TO use in the cloud, ahve logic that looks for local file, if not present, try to download with ipynb cloud !bash notation and if it fails (for reasons or because user is local) just prompt to download from URL.

In [27]:
r["load"]("CellChatDB.mouse.rda")

ccDB_mouse = r["CellChatDB.mouse"]
ccDB_mouse[0]

,interaction_name,pathway_name,ligand,receptor,agonist,antagonist,co_A_receptor,co_I_receptor,evidence,annotation,interaction_name_2
TGFB1_TGFBR1_TGFBR2,TGFB1_TGFBR1_TGFBR2,TGFb,Tgfb1,TGFbR1_R2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,KEGG: mmu04350,Secreted Signaling,Tgfb1 - (Tgfbr1+Tgfbr2)
TGFB2_TGFBR1_TGFBR2,TGFB2_TGFBR1_TGFBR2,TGFb,Tgfb2,TGFbR1_R2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,KEGG: mmu04350,Secreted Signaling,Tgfb2 - (Tgfbr1+Tgfbr2)
TGFB3_TGFBR1_TGFBR2,TGFB3_TGFBR1_TGFBR2,TGFb,Tgfb3,TGFbR1_R2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,KEGG: mmu04350,Secreted Signaling,Tgfb3 - (Tgfbr1+Tgfbr2)
TGFB1_ACVR1B_TGFBR2,TGFB1_ACVR1B_TGFBR2,TGFb,Tgfb1,ACVR1B_TGFbR2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,PMID: 27449815,Secreted Signaling,Tgfb1 - (Acvr1b+Tgfbr2)
TGFB1_ACVR1C_TGFBR2,TGFB1_ACVR1C_TGFBR2,TGFb,Tgfb1,ACVR1C_TGFbR2,TGFb agonist,TGFb antagonist,,TGFb inhibition receptor,PMID: 27449815,Secreted Signaling,Tgfb1 - (Acvr1c+Tgfbr2)
...,...,...,...,...,...,...,...,...,...,...,...
TIGIT_PVR,TIGIT_PVR,TIGIT,Tigit,Pvr,,,,,PMID:24987108,Cell-Cell Contact,Tigit - Pvr
ITGA4_ITGB1_VCAM1,ITGA4_ITGB1_VCAM1,VCAM,ITGA4_ITGB1,Vcam1,,,,,KEGG: mmu04514,Cell-Cell Contact,Vcam1 - (Itga4+Itgb1)
ITGA9_ITGB1_VCAM1,ITGA9_ITGB1_VCAM1,VCAM,ITGA9_ITGB1,Vcam1,,,,,KEGG: mmu04514,Cell-Cell Contact,Vcam1 - (Itga9+Itgb1)
ITGA4_ITGB7_VCAM1,ITGA4_ITGB7_VCAM1,VCAM,ITGA4_ITGB7,Vcam1,,,,,KEGG: mmu04514,Cell-Cell Contact,Vcam1 - (Itga4+Itgb7)


In [28]:
ccDB_mouse[3]

,Symbol,Name,EntrezGene.ID,Ensembl.Gene.ID,HomoloGene.ID,HGNC.ID
MGI:87853,a,nonagouti,50518,ENSMUSG00000027596,1264,HGNC:745
MGI:87854,Pzp,"PZP, alpha-2-macroglobulin like",11287,ENSMUSG00000030359,104112,null
MGI:87859,Abl1,"c-abl oncogene 1, non-receptor tyrosine kinase",11350,ENSMUSG00000026842,3783,HGNC:76
MGI:87860,Abl2,v-abl Abelson murine leukemia viral oncogene 2...,11352,ENSMUSG00000026596,5278,HGNC:77
MGI:87862,Scgb1b27,"secretoglobin, family 1B, member 27",11354,ENSMUSG00000066583,114479,null
...,...,...,...,...,...,...
MGI:1913728,Cd99,CD99 antigen,673094,,48107,HGNC:7082
MGI:96876,Klra,"killer cell lectin-like receptor, subfamily A",17055,,,HGNC:6372
MGI:1891386,Ccl21c,chemokine (C-C motif) ligand 21C (leucine),111654,,,HGNC:10620
MGI:107670,Ifna,interferon alpha,4663,,,
